## PDF to CSV (New)

### Test

In [ ]:
### Test

import pdfplumber
import pandas as pd

# Path to your PDF file
pdf_path = "/Volumes/Ping Pong/TS2/00194909_f_20181023143245.pdf"


# Open the PDF file using pdfplumber
with pdfplumber.open(pdf_path) as pdf:
    start_page = None
    dfs = []

    # Iterate over each page in the PDF
    for i, page in enumerate(pdf.pages):
        page_text = page.extract_text()
        
        # Check if "1. Location Parameters" is present in the page text
        if "1. Location Parameters" in page_text:
            start_page = i
            break

    if start_page is not None:
        # Read the PDF file again, starting from the identified page
        for page in pdf.pages[start_page:]:
            tables = page.extract_tables()
            for table in tables:
                # Generate unique column names for each table
                columns = [f"Column_{i}" for i in range(len(table[0]))]
                df = pd.DataFrame(table[0:], columns=columns)
                dfs.append(df)

        if dfs:
            # Concatenate all tables into a single DataFrame
            combined_df = pd.concat(dfs, ignore_index=True)

            # Shift first three rows to column 2
            combined_df.loc[0:3, 'Column_1'] = combined_df['Column_0']
            
            # Drop unnecessary columns
            combined_df = combined_df.drop(columns=["Column_0","Column_3", "Column_7", "Column_8"])

            # Drop empty rows
            combined_df = combined_df.dropna(how='all')

            # Remove line breaks
            combined_df = combined_df.replace('\n', ' ', regex=True)

            # Save the combined DataFrame as a CSV file
            output_csv = "output.csv"
            combined_df.to_csv(output_csv, header=None,index=False)
            print(f"Tables saved in {output_csv}")
        else:
            print("No tables found in the PDF.")
    else:
        print("Unable to find '1. Location Parameters' in the PDF.")

# print(combined_df)


In [ ]:
### Loop Test

import pandas as pd
import os
import glob

## Change folder path and output_csv for each run
# Path to the folder containing the files
folder_path = "/Users/sid/Documents/MA/AN/"

# Get a list of files in the folder
file_list = glob.glob(os.path.join(folder_path, "*.csv"))

# Initialize an empty DataFrame to store the combined results
final_df = pd.DataFrame()

# Initialize a dictionary to store the shape counts
shape_counts = {}

# Loop over each file
for file_path in file_list:
    # Read the file into a DataFrame
    df2 = pd.read_csv(file_path)

    ## State info
    df4 = df2.drop(df2.index[3:], inplace=False) # only keep required rows
    df4 = df4.iloc[:, [0, 1]] # drops unnecesary columns
    df4 = df4.transpose().reset_index()
    df4.columns = df4.iloc[0]
    df4 = df4[1:]  # Remove the header row

    ## District info
    df5 = df2.drop(df2.index[2:], inplace=False)
    df5 = df5.iloc[:, [2, 4]]
    df5 = df5.transpose().reset_index()
    df5.columns = df5.iloc[0]
    df5 = df5[1:]  # Remove the header row

    ## Village Parameters
    df3 = df2.iloc[4:].transpose()
    # Extract the header row and set it as the column names
    df3.columns = df3.iloc[0]
    df3 = df3[1:]  # Remove the header row
    df3 = df3.iloc[1:]

    df3 = df3.reset_index(drop=True)

    # # Remove row 1,3
    rows_to_drop = [0,2] 
    df3 = df3.drop(rows_to_drop)
    df3 = df3.fillna(0)

    # Reset the index
    # df3 = df3.reset_index(drop=True)
    # df3 = df3.dropna(how='all')
    combined_df = pd.concat([df4, df5,df3], axis=1)
    # print(combined_df.shape[1])

    # Get the shape of combined_df
    df_shape = combined_df.shape

    # Increment the count for the shape in the dictionary
    if df_shape in shape_counts:
        shape_counts[df_shape] += 1
    else:
        shape_counts[df_shape] = 1

    if combined_df.shape[1] == 54:
        final_df = final_df.append(combined_df, ignore_index=True)
     
output_csv = "/Users/sid/Documents/MA_20/AN.csv"
final_df.to_csv(output_csv, index=False)
print(f"Combined results saved in {output_csv}")

# Print the shape counts
print("Shape Counts:")
for shape, count in shape_counts.items():
    print(f"Shape: {shape}, Count: {count}")
    

### PDF to CSV loop

#### 2020 Data

In [ ]:
import glob
import os
import pdfplumber
import pandas as pd

# Folder path containing the PDF files
folder_path = "/Users/sid/Documents/MA/TN/"

# Output folder path for saving CSV files
output_folder = "/Users/sid/Documents/MA_20/TN/"

# Get a list of PDF files in the folder
pdf_files = glob.glob(folder_path + "*.pdf")

# Function to shift data by 2 columns if Column 3 is not empty. Some pdfs formatting causes page 3 data to be misinterpretted. 
def shift_data(row):
    if row.name >= 30 and pd.notnull(row[3]):  # Check if row index is >= 30 and Column 3 is not empty
        row[5:] = row[3:].values[:-2]  # Shift the data in columns 3 and onward by 2 positions
    return row

# Loop over each PDF file
for pdf_path in pdf_files:
    # print("Processing:", pdf_path)

    # Open the PDF file using pdfplumber
    with pdfplumber.open(pdf_path) as pdf:
        start_pages = []
        dfs = []

        # Iterate over each page in the PDF
        for i, page in enumerate(pdf.pages):
            page_text = page.extract_text()

            # Check if "1. Location Parameters" is present in the page text
            if "1. Location Parameters" in page_text:
                start_pages.append(i)

        if start_pages:
            # Add the last page of the PDF as an end point to the list of start pages
            # This helps handle the case where the last "1. Location Parameters" occurs at the end of the PDF
            start_pages.append(len(pdf.pages))

            # Iterate over each start page
            for j in range(len(start_pages) - 1):
                temp_dfs = []
                start_page = start_pages[j]
                end_page = start_pages[j + 1]

                # Read the PDF file again, starting from the identified start page and ending at the next "1. Location Parameters" page
                for page in pdf.pages[start_page:end_page]:
                    tables = page.extract_tables()
                    for table in tables:
                        # Generate unique column names for each table
                        columns = [f"Column_{i}" for i in range(len(table[0]))]
                        df = pd.DataFrame(table[0:], columns=columns)
                        temp_dfs.append(df)

                if temp_dfs:
                    # Concatenate all tables into a single DataFrame
                    combined_df = pd.concat(temp_dfs, ignore_index=True)

                    combined_df = combined_df.apply(shift_data, axis=1)

                    combined_df = combined_df.drop(combined_df.columns[[4, 5]], axis=1)

                    # Shift first three rows to column 2
                    # combined_df.loc[0:3, 'Column_1'] = combined_df['Column_0']

                    # Drop unnecessary columns.
                    # combined_df = combined_df.drop(columns=["Column_0", "Column_3"]) #, "Column_7", "Column_8"

                    # Drop empty rows
                    # combined_df = combined_df.dropna(how='all')

                    # Remove line breaks
                    combined_df = combined_df.replace('\n', ' ', regex=True)

                    # Generate a unique output filename using a counter
                    filename = os.path.splitext(os.path.basename(pdf_path))[0]  # Get the base filename of the PDF
                    counter = 1
                    while True:
                        output_csv = os.path.join(output_folder, f"{filename}_output_{counter}.csv")
                        if not os.path.exists(output_csv):
                            break
                        counter += 1

                    output_csv = os.path.join(output_folder, f"{filename}_output_{j + 1}.csv")
                    
                    # Save the combined DataFrame as a CSV file
                    combined_df.to_csv(output_csv, header=None, index=False)
                    # print(f"Tables saved in {output_csv}")
                else:
                    print("No tables found in the PDF.")
        else:
            print("Unable to find '1. Location Parameters' in the PDF.")


#### 2018 Data

In [ ]:
import glob
import os
import pdfplumber
import pandas as pd

# Folder path containing the PDF files
folder_path = "/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/MA/MA_18_pdf/NG/"

# Output folder path for saving CSV files
output_folder = "/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/MA/MA_18/NG/"

# Get a list of PDF files in the folder
pdf_files = glob.glob(folder_path + "*.pdf")

# Function to shift data by 2 columns if Column 3 is not empty. Some pdfs formatting causes page 3 data to be misinterpretted. 
def shift_data(row):
    if row.name >= 30 and pd.notnull(row[3]):  # Check if row index is >= 30 and Column 3 is not empty
        row[5:] = row[3:].values[:-2]  # Shift the data in columns 3 and onward by 2 positions
    return row

# Get total number of PDF files
total_pdfs = len(pdf_files)
processed_pdfs = 0

# Define the intervals at which to print progress notifications
progress_intervals = [20, 40, 60, 80, 100]
current_interval = 0


# Loop over each PDF file
for pdf_path in pdf_files:
    # print("Processing:", pdf_path)

    # Open the PDF file using pdfplumber
    with pdfplumber.open(pdf_path) as pdf:
        start_pages = []
        dfs = []

        # Iterate over each page in the PDF
        for i, page in enumerate(pdf.pages):
            page_text = page.extract_text()

            # Check if "1. Location Parameters" is present in the page text
            if "1. Location Parameters" in page_text:
                start_pages.append(i)

        if start_pages:
            # Add the last page of the PDF as an end point to the list of start pages
            # This helps handle the case where the last "1. Location Parameters" occurs at the end of the PDF
            start_pages.append(len(pdf.pages))

            # Iterate over each start page
            for j in range(len(start_pages) - 1):
                temp_dfs = []
                start_page = start_pages[j]
                end_page = start_pages[j + 1]

                # Read the PDF file again, starting from the identified start page and ending at the next "1. Location Parameters" page
                for page in pdf.pages[start_page:end_page]:
                    tables = page.extract_tables()
                    for table in tables:
                        # Generate unique column names for each table
                        columns = [f"Column_{i}" for i in range(len(table[0]))]
                        df = pd.DataFrame(table[0:], columns=columns)
                        temp_dfs.append(df)

                if temp_dfs:
                    # Concatenate all tables into a single DataFrame
                    combined_df = pd.concat(temp_dfs, ignore_index=True)

                    combined_df = combined_df.apply(shift_data, axis=1)

                    combined_df = combined_df.drop(combined_df.columns[[7, 8]], axis=1)

                    # Shift first three rows to column 2
                    # combined_df.loc[0:3, 'Column_1'] = combined_df['Column_0']

                    # Drop unnecessary columns.
                    # combined_df = combined_df.drop(columns=["Column_0", "Column_3"]) #, "Column_7", "Column_8"

                    # Drop empty rows
                    # combined_df = combined_df.dropna(how='all')

                    # Remove line breaks
                    combined_df = combined_df.replace('\n', ' ', regex=True)

                    # Generate a unique output filename using a counter
                    filename = os.path.splitext(os.path.basename(pdf_path))[0]  # Get the base filename of the PDF
                    counter = 1
                    while True:
                        output_csv = os.path.join(output_folder, f"{filename}_output_{counter}.csv")
                        if not os.path.exists(output_csv):
                            break
                        counter += 1

                    output_csv = os.path.join(output_folder, f"{filename}_output_{j + 1}.csv")
                    
                    # Save the combined DataFrame as a CSV file
                    combined_df.to_csv(output_csv, header=None, index=False)
                    # print(f"Tables saved in {output_csv}")
                    #     # Increment the processed PDFs counter
                    # processed_pdfs += 1
                    
                    # # Calculate the progress percentage
                    # progress_percentage = (processed_pdfs / total_pdfs) * 100
                    
                    # # Check if the current progress is greater than or equal to the next interval
                    # if progress_percentage >= progress_intervals[current_interval]:
                    #     # Print progress notification
                    #     print(f"{progress_percentage:.2f}% of PDFs converted to CSVs")
                    #     # Move to the next interval
                    #     current_interval += 1
                        
                    # # Check if all intervals have been reached
                    # if current_interval >= len(progress_intervals):
                    #     break
                else:
                    print("No tables found in the PDF.")
        else:
            print("Unable to find '1. Location Parameters' in the PDF.")



### CSV Manipulation

#### Completed correct code below:

In [ ]:
import pandas as pd
import os
import glob

## Change folder path and output_csv for each run
# Path to the folder containing the files
folder_path = "/Users/sid/Documents/MA_20/AN/"

# Get a list of files in the folder
file_list = glob.glob(os.path.join(folder_path, "*.csv"))

# Initialize an empty DataFrame to store the combined results
final_df = pd.DataFrame()

# Initialize a dictionary to store the shape counts
shape_counts = {}

# Loop over each file
for file_path in file_list:
    # Read the file into a DataFrame
    df2 = pd.read_csv(file_path)

    ## State info
    df4 = df2.drop(df2.index[3:], inplace=False) # only keep required rows
    df4 = df4.iloc[:, [0, 1]] # drops unnecesary columns
    df4 = df4.transpose().reset_index()
    df4.columns = df4.iloc[0]
    df4 = df4[1:]  # Remove the header row
    df4 = df4.reset_index(drop=True)

    ## District info
    df5 = df2.drop(df2.index[2:], inplace=False)
    df5 = df5.iloc[:, [2, 3]] ## changed index, different from TS
    df5 = df5.transpose().reset_index()
    df5.columns = df5.iloc[0]
    df5 = df5[1:]  # Remove the header row
    df5 = df5.reset_index(drop=True)

    ## Village Parameters
    df3 = df2.drop(df2.columns[0], axis=1) ## Added this, different from TS code
    df3 = df3.iloc[8:].transpose() ## if TS, df3 = df2.iloc
    df3 = df3.dropna(axis=1, how='all')  ## Added this, different from TS code
    # Extract the header row and set it as the column names
    df3.columns = df3.iloc[0]
    df3 = df3[1:]  # Remove the header row
    df3 = df3.iloc[1:]

    df3 = df3.reset_index(drop=True)

    # # # Remove row 1,3
    # rows_to_drop = [0,2] 
    # df3 = df3.drop(rows_to_drop)
    # df3 = df3.fillna(0)

    # Reset the index
    # df3 = df3.reset_index(drop=True)
    # df3 = df3.dropna(how='all')
    combined_df = pd.concat([df4, df5,df3], axis=1)
    # print(combined_df.shape[1])

    # Get the shape of combined_df
    df_shape = combined_df.shape

    # Increment the count for the shape in the dictionary
    if df_shape in shape_counts:
        shape_counts[df_shape] += 1
    else:
        shape_counts[df_shape] = 1

    # if combined_df.shape[1] == 54: ## Only use in 2018 data
    final_df = final_df.append(combined_df, ignore_index=True)
     
output_csv = "/Users/sid/Documents/MA_20/AN.csv"
final_df.to_csv(output_csv, index=False)
print(f"Combined results saved in {output_csv}")

# Print the shape counts
print("Shape Counts:")
for shape, count in shape_counts.items():
    print(f"Shape: {shape}, Count: {count}")
    

#### Master Folder loop code below:

In [ ]:
import pandas as pd
import os
import glob
import time

# Path to the master folder containing subfolders
master_folder = "/Users/sid/Documents/MA_20/"

# Get a list of subfolders in the master folder
subfolders = [subfolder for subfolder in os.listdir(master_folder) if os.path.isdir(os.path.join(master_folder, subfolder))]

# Loop through each subfolder
for subfolder in subfolders:
    if subfolder != "UP":
        print(f"Skipping folder {subfolder}.\n")
        continue  # Skip processing this folder and move to the next

    folder_path = os.path.join(master_folder, subfolder)
    
    # Get a list of files in the subfolder
    file_list = glob.glob(os.path.join(folder_path, "*.csv"))

    print(f"Beginning folder {subfolder}.\n")

    start_time = time.time()

    final_df = pd.DataFrame()
    shape_counts = {}
    
    for file_path in file_list:
        df2 = pd.read_csv(file_path)

        ## State info
        df4 = df2.drop(df2.index[3:], inplace=False) # only keep required rows
        df4 = df4.iloc[:, [0, 1]] # drops unnecesary columns
        df4 = df4.transpose().reset_index()
        df4.columns = df4.iloc[0]
        df4 = df4[1:]  # Remove the header row
        df4 = df4.reset_index(drop=True)

        ## District info
        df5 = df2.drop(df2.index[2:], inplace=False)
        df5 = df5.iloc[:, [2, 3]] ## changed index, different from TS
        df5 = df5.transpose().reset_index()
        df5.columns = df5.iloc[0]
        df5 = df5[1:]  # Remove the header row
        df5 = df5.reset_index(drop=True)

        ## Village Parameters
        df3 = df2.drop(df2.columns[0], axis=1) ## Added this, different from TS code
        df3 = df3.iloc[8:].transpose() ## if TS, df3 = df2.iloc
        df3 = df3.dropna(axis=1, how='all')  ## Added this, different from TS code
        # Extract the header row and set it as the column names
        df3.columns = df3.iloc[0]
        df3 = df3[1:]  # Remove the header row
        df3 = df3.iloc[1:]

        df3 = df3.reset_index(drop=True)

        combined_df = pd.concat([df4, df5,df3], axis=1)

        # Get the shape of combined_df
        df_shape = combined_df.shape

        # Increment the count for the shape in the dictionary
        if df_shape in shape_counts:
            shape_counts[df_shape] += 1
        else:
            shape_counts[df_shape] = 1

        # if combined_df.shape[1] == 54: ## Only use in 2018 data
        final_df = final_df.append(combined_df, ignore_index=True)

    # Calculate time taken
    end_time = time.time()
    time_taken = end_time - start_time
    
    # Save combined results to CSV
    output_csv = os.path.join(master_folder, f"{subfolder}.csv")
    final_df.to_csv(output_csv, index=False)
    print(f"Combined results for {subfolder} saved in {output_csv}")

    # Print shape counts
    print("Shape Counts:")
    for shape, count in shape_counts.items():
        print(f"Shape: {shape}, Count: {count}")

    print(f"Folder {subfolder} completed in {time_taken:.2f} seconds.\n")


In [ ]:
import pandas as pd
import os

# Specify the directory containing CSV files
csv_directory = "/Users/sid/Documents/MA_20/"

# Get a list of all CSV files in the directory
csv_files = [file for file in os.listdir(csv_directory) if file.endswith(".csv")]

# Dictionary to store column sets
column_sets = {}

# Loop through each CSV file
for csv_file in csv_files:
    file_path = os.path.join(csv_directory, csv_file)
    
    # Read the CSV file using pandas
    df = pd.read_csv(file_path)
    
    # Get the shape of the DataFrame
    shape = df.shape
    
    # Get the set of columns
    columns = tuple(df.columns)
    
    # Store the columns and shape in the dictionary
    column_sets[file_path] = (shape, columns)

# Create a dictionary to group files with same columns
column_groups = {}

# Group files with same columns
for file, (shape, columns) in column_sets.items():
    columns_key = tuple(columns)
    
    if columns_key in column_groups:
        column_groups[columns_key].append((file, shape))
    else:
        column_groups[columns_key] = [(file, shape)]

# Print grouped information
for columns_key, files_and_shapes in column_groups.items():
    if len(files_and_shapes) > 1:
        file_info = [f"{os.path.basename(file)} ({shape})" for file, shape in files_and_shapes]
        print(f"Files {', '.join(file_info)} have the same columns.")
    else:
        file_info = files_and_shapes[0]
        print(f"File {os.path.basename(file_info[0])} ({file_info[1]}) has different columns.")

print("Comparison finished.")


In [ ]:
import pandas as pd
import os

# Specify the directory containing CSV files
csv_directory = "/Users/sid/Documents/MA_20/"

# Get a list of all CSV files in the directory
csv_files = [file for file in os.listdir(csv_directory) if file.endswith(".csv")]

# Dictionary to store column sets
column_sets = {}

# Loop through each CSV file
for csv_file in csv_files:
    file_path = os.path.join(csv_directory, csv_file)
    
    # Read the CSV file using pandas
    df = pd.read_csv(file_path)
    
    # Get the shape of the DataFrame
    shape = df.shape
    
    # Get the set of columns
    columns = tuple(df.columns)
    
    # Store the columns and shape in the dictionary
    column_sets[file_path] = (shape, columns)

# Create a dictionary to group files with same columns
column_groups = {}

# Group files with same columns
for file, (shape, columns) in column_sets.items():
    columns_key = tuple(columns)
    
    if columns_key in column_groups:
        column_groups[columns_key].append((file, shape))
    else:
        column_groups[columns_key] = [(file, shape)]

# Concatenate files with similar columns
output_counter = 1
for columns_key, files_and_shapes in column_groups.items():
    if len(files_and_shapes) > 1:
        # Extract paths of files with same columns
        file_paths = [file for file, shape in files_and_shapes]
        
        # Read and concatenate the dataframes
        dfs = [pd.read_csv(file_path) for file_path in file_paths]
        combined_df = pd.concat(dfs, ignore_index=True)
        
        # Create an output file name using incremental numbering
        output_file = os.path.join(csv_directory, f"{output_counter}.csv")
        
        # Save the combined dataframe to the output file name
        combined_df.to_csv(output_file, index=False)
        print(f"Combined files with {columns_key} columns saved as {output_file}")
        
        output_counter += 1

print("Comparison and concatenation finished.")
